# Install dependencies

* Use `pip` to install dependencies.
* Install `graphviz` to render decision trees.




In [1]:
!pip install --upgrade setuptools jedi
!pip install matplotlib numpy pandas scikit-learn==1.2.2 scipy trustee==1.1.4
!apt -qqq install graphviz

  Using cached setuptools-68.1.2-py3-none-any.whl (805 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 57.5.0
    Uninstalling setuptools-57.5.0:
      Successfully uninstalled setuptools-57.5.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
trustee 1.1.4 requires setuptools<58.0.0,>=57.0.0, but you have setuptools 68.1.2 which is incompatible.


  Using cached setuptools-57.5.0-py3-none-any.whl (819 kB)
  Attempting uninstall: setuptools
    Found existing installation: setuptools 68.1.2
    Uninstalling setuptools-68.1.2:
      Successfully uninstalled setuptools-68.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
arviz 0.15.1 requires setuptools>=60.0.0, but you have setuptools 57.5.0 which is incompatible.
cvxpy 1.3.2 requires setuptools>65.5.1, but you have setuptools 57.5.0 which is incompatible.


# Traditional ML Pipeline

* Read CIC-IDS-2017 dataset
  * The CIC-IDS-2017 dataset is hosted in the Github repo from the project.

* To read the data, we are using a helper function from the Trustee package.
  * This method automatically one-hot encodes the categorical variables of the dataset, based on the provided metadata.

In [3]:
import numpy as np
from trustee.utils import dataset
from trustee.utils.const import CIC_IDS_2017_DATASET_META

import warnings
warnings.filterwarnings("ignore")

DF_PATH = "https://github.com/TrusteeML/emperor/raw/main/use_cases/heartbleed_case/res/dataset/CIC-IDS-2017_OverSampled_min.csv.zip"

# if using oversampled df
CIC_IDS_2017_DATASET_META["is_dir"] = False

# Step 1: Parse train-test def
X, y, feature_names, _, _ = dataset.read(
    DF_PATH, metadata=CIC_IDS_2017_DATASET_META, as_df=True, verbose=True
)

# select subsample of the dataset
idx = np.random.permutation(X.index)
idx = idx[:int(len(idx) * 0.1)]

X = X.loc[idx]
y = y.loc[idx]

========== Metadata start. ==========
Names: ['Destination Port', 'Flow Duration', 'Total Fwd Packets', 'Total Backward Packets', 'Total Length of Fwd Packets', 'Total Length of Bwd Packets', 'Fwd Packet Length Max', 'Fwd Packet Length Min', 'Fwd Packet Length Mean', 'Fwd Packet Length Std', 'Bwd Packet Length Max', 'Bwd Packet Length Min', 'Bwd Packet Length Mean', 'Bwd Packet Length Std', 'Flow Bytes/s', 'Flow Packets/s', 'Flow IAT Mean', 'Flow IAT Std', 'Flow IAT Max', 'Flow IAT Min', 'Fwd IAT Total', 'Fwd IAT Mean', 'Fwd IAT Std', 'Fwd IAT Max', 'Fwd IAT Min', 'Bwd IAT Total', 'Bwd IAT Mean', 'Bwd IAT Std', 'Bwd IAT Max', 'Bwd IAT Min', 'Fwd PSH Flags', 'Fwd URG Flags', 'Bwd Header Length', 'Fwd Packets/s', 'Bwd Packets/s', 'Min Packet Length', 'Max Packet Length', 'Packet Length Mean', 'Packet Length Std', 'Packet Length Variance', 'FIN Flag Count', 'SYN Flag Count', 'RST Flag Count', 'PSH Flag Count', 'ACK Flag Count', 'URG Flag Count', 'CWE Flag Count', 'ECE Flag Count', 'Down/U

Split dataset into train and test sets

In [4]:
import numpy as np
from sklearn.model_selection import train_test_split

X_indexes = np.arange(0, X.shape[0])
X_train, X_test, y_train, y_test = train_test_split(X_indexes, y, train_size=0.7, stratify=y)
X_train = X.iloc[X_train]
X_test = X.iloc[X_test]

Train Random Forest Classifier

In [5]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report

blackbox = RandomForestClassifier(n_jobs=4)
blackbox.fit(X_train, y_train)

y_pred = blackbox.predict(X_test)

print("Blackbox model classification report with IID:")
print(
    "\n{}".format(
        classification_report(
            y_test,
            y_pred,
            digits=3,
            target_names=CIC_IDS_2017_DATASET_META["classes"],
        )
    )
)


Blackbox model classification report with IID:

                          precision    recall  f1-score   support

                  BENIGN      0.981     0.983     0.982       689
                     Bot      0.993     0.994     0.993       680
                    DDoS      0.999     1.000     0.999       674
           DoS GoldenEye      0.996     0.996     0.996       675
                DoS Hulk      1.000     0.994     0.997       665
        DoS Slowhttptest      0.997     0.990     0.993       690
           DoS slowloris      0.993     0.994     0.993       675
             FTP-Patator      1.000     1.000     1.000       692
              Heartbleed      1.000     1.000     1.000       663
            Infiltration      1.000     1.000     1.000       672
                PortScan      0.999     1.000     0.999       684
             SSH-Patator      1.000     0.999     0.999       674
  Web Attack Brute Force      0.960     0.900     0.929       689
Web Attack Sql Injection   

# Trustee
Run Classification Trustee on trained Random Forest Classifier



In [6]:
import graphviz

from sklearn import tree
from sklearn import datasets

from trustee import ClassificationTrustee

# Initialize Trustee and fit for classification models
trustee = ClassificationTrustee(expert=blackbox)
trustee.fit(X_train, y_train, num_iter=30, num_stability_iter=2, samples_size=0.3, verbose=True)

# Get the best explanation from Trustee
dt, pruned_dt, agreement, reward = trustee.explain(top_k=10)
print(f"Model explanation training (agreement, fidelity): ({agreement}, {reward})")
print(f"Model Explanation size: {dt.tree_.node_count}")
print(f"Top-k Prunned Model explanation size: {pruned_dt.tree_.node_count}")

Initializing training dataset using RandomForestClassifier(n_jobs=4) as expert model
Expert model score: 1.0
Initializing Trustee outer-loop with 2 iterations
########## Outer-loop Iteration 0/2 ##########
Initializing Trustee inner-loop with 2 iterations
########## Inner-loop Iteration 0/30 ##########
Sampling 5011 points from training dataset with (16706, 16706) entries
Student model 0-0 trained with depth 25 and 207 leaves:
Student model score: 0.9488792648902211
Student model 0-0 fidelity: 0.9488792648902211
########## Inner-loop Iteration 1/30 ##########
Sampling 5011 points from training dataset with (18210, 18210) entries
Student model 0-1 trained with depth 25 and 186 leaves:
Student model score: 0.940621548133763
Student model 0-1 fidelity: 0.940621548133763
########## Inner-loop Iteration 2/30 ##########
Sampling 5011 points from training dataset with (19714, 19714) entries
Student model 0-2 trained with depth 26 and 210 leaves:
Student model score: 0.9523335980523702
Student

Evaluate explanations produced by Trustee

In [7]:
# Use explanations to make predictions
dt_y_pred = dt.predict(X_test)
pruned_dt_y_pred = pruned_dt.predict(X_test)

# Evaluate accuracy and fidelity of explanations
print("Model explanation global fidelity report:")
print(classification_report(y_pred, dt_y_pred))
print("Top-k Model explanation global fidelity report:")
print(classification_report(y_pred, pruned_dt_y_pred))

print("Model explanation score report:")
print(classification_report(y_test, dt_y_pred))
print("Top-k Model explanation score report:")
print(classification_report(y_test, pruned_dt_y_pred))

Model explanation global fidelity report:
              precision    recall  f1-score   support

           0       0.97      0.90      0.93       690
           1       0.98      1.00      0.99       681
           2       1.00      1.00      1.00       675
           3       0.98      0.97      0.97       675
           4       0.98      0.98      0.98       661
           5       0.98      0.98      0.98       685
           6       0.94      0.97      0.96       676
           7       1.00      1.00      1.00       692
           8       1.00      1.00      1.00       663
           9       1.00      1.00      1.00       672
          10       0.99      0.98      0.99       685
          11       1.00      1.00      1.00       673
          12       0.67      0.68      0.68       646
          13       0.98      1.00      0.99       691
          14       0.72      0.70      0.71       764

    accuracy                           0.94     10229
   macro avg       0.94      0.94     

Render Decision Tree explanations

In [8]:

# Output decision tree to pdf
dot_data = tree.export_graphviz(
    dt,
    class_names=CIC_IDS_2017_DATASET_META["classes"],
    feature_names=feature_names,
    filled=True,
    rounded=True,
    special_characters=True,
)
graph = graphviz.Source(dot_data)
graph.render("dt_explanation")

# Output pruned decision tree to pdf
dot_data = tree.export_graphviz(
    pruned_dt,
    class_names=CIC_IDS_2017_DATASET_META["classes"],
    feature_names=feature_names,
    filled=True,
    rounded=True,
    special_characters=True,
)
graph = graphviz.Source(dot_data)
graph.render("pruned_dt_explation")

'pruned_dt_explation.pdf'

# Validation

Read Validation dataset with Out-of-Distribution samples

In [17]:
VALIDATION_DF_PATH = "https://raw.githubusercontent.com/TrusteeML/emperor/main/use_cases/heartbleed_case/res/dataset/validation/heartbleed.csv"

X_validate, y_validate, _, _, _ = dataset.read(VALIDATION_DF_PATH, metadata=CIC_IDS_2017_DATASET_META, as_df=True)


Use Validation dataset to evaluate trained Random Forest Classifier


In [18]:
y_val_pred = blackbox.predict(X_validate)

print("Blackbox model classification report with OOD:")
print(
    "\n{}".format(
        classification_report(
            y_validate,
            y_val_pred,
            digits=3,
            target_names=["BENIGN", "Heartbleed"],
        )
    )
)

Blackbox model classification report with OOD:

              precision    recall  f1-score   support

      BENIGN      0.000     0.000     0.000       0.0
  Heartbleed      0.000     0.000     0.000    1000.0

    accuracy                          0.000    1000.0
   macro avg      0.000     0.000     0.000    1000.0
weighted avg      0.000     0.000     0.000    1000.0



# Trust Report

Run Trust Report on trained Random Forest Classifier

In [15]:
from trustee.report.trust import TrustReport

trust_report = TrustReport(
    blackbox,
    X=X,
    y=y,
    top_k=10,
    max_iter=3,
    trustee_num_iter=3,
    num_pruning_iter=3,
    trustee_num_stability_iter=3,
    trustee_sample_size=0.30,
    analyze_stability=True,
    skip_retrain=False,
    feature_names=feature_names,
    class_names=CIC_IDS_2017_DATASET_META["classes"],
    verbose=True,
)

print(trust_report)

Running Trust Report...
Preparing data...
Splitting dataset for training and testing...
X size: 34096; y size: 34096
Done!
Done!
Progress |█████-----------------------------------------------------------------------------------------------| 5.3% Complete
Done!
Progress |██████████------------------------------------------------------------------------------------------| 10.5% Complete
Fitting blackbox model...
Done!
Blackbox model score report with training data:

              precision    recall  f1-score   support

           0      0.989     0.979     0.984       703
           1      0.993     1.000     0.996       687
           2      0.998     0.998     0.998       638
           3      0.995     0.998     0.997       656
           4      0.989     0.998     0.994       638
           5      0.999     0.994     0.996       674
           6      0.997     0.999     0.998       702
           7      1.000     1.000     1.000       693
           8      1.000     1.000     1.000 

In [19]:
import warnings
warnings.filterwarnings("ignore")

OUTPUT_PATH = "res/output"
trust_report.save(OUTPUT_PATH)

# warning: execution time ~ 30 minutes

Saving...


AttributeError: ignored

In [ ]:
#